In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
from matplotlib.pyplot import figure
from collections import Counter
from scipy.stats import spearmanr,pearsonr,fisher_exact,binom_test
import scanpy as sc

In [ ]:
#Colors
mouse = "#F2C911"
rat = "#65B0AC"
intrinsic = "#F55F00"
extrinsic = "#7D9AF4"
reinforcing = "#9B00F5"
opposing = "#F50901"
interaction = "#1E771A"

sns.set(font_scale=1.5)
sns.set_style("white")
d_ct_abrev = {"Forebrain glutamatergic progenitors":"brain.glut.prog", "Forebrain GABAergic progenitors":"brain.GABA.prog", "Intermediate progenitors":"inter.prog", "Forebrain glutamatergic neurons":"brain.glut.neu", "Forebrain GABAergic neurons":"brain.GABA.neu", "Spinal GABAergic neurons":"spine.GABA.neu", "Spinal glutamatergic neurons":"spine.glut.neu", "Chondrocytes":"chondrocyte", "Mesenchyme 0":"mesen.0", "Mesenchyme 2":"mesen.2", "Mesenchyme cycling":"mesen.cyc", "Forebrain GABAergic neurons 0":"Toss"}

def file_to_celltype(x):
    x = x.split("/")[-1]
    x = x.replace("_GO_Molecular_Function_2023", "").replace("_GO_Biological_Process_2023", "").replace("Mesechyme", "Mesenchyme").replace("Brain_Or_2010_Div_New4_NewNorm_", "")
    x = x.replace("Mesenchymal_Or_2010_Div_New4_NewNorm_", "").replace("Chondrocyte_Or_2010_Div_New4_NewNorm_", "").replace(".csv", "").replace("_", " ").replace(" all", "")
    x = x.replace("Glutamatergic", "Forebrain glutamatergic").replace("GABAergic", "Forebrain GABAergic").replace("Spinal Forebrain", "Spinal").replace("Chondrocytes", "Chondrocyte")
    return x

plt.rcParams["font.family"] = "Arial"

In [ ]:
#Commented and uncommented this to get the right file and cell type

excit = [0, 5]
mes_0 = [0]
inh_prog = [2]
inhib_all = [0, 1, 3, 4]
spinal_excit = [6]
#brain = sc.read("Final/Brain_Subclustered_Prelim.h5ad")
#tp = brain[brain.obs["leiden"].isin([str(x) for x in excit])]

#chondro = sc.read("Final/Chondrocyte_Subclustered_Prelim.h5ad")
#tp = chondro[chondro.obs["leiden"].isin([str(0)])]

#mes = sc.read("Final/Mesenchymal_Subclustered_Prelim.h5ad")
#tp = mes[mes.obs["leiden"].isin([str(0)])]

inh = sc.read("Final/GABAergic_Forebrain.h5ad")
tp = inh[inh.obs["leiden"].isin([str(x) for x in inhib_all])]


In [ ]:
#Changed tissue as appropriate

tissue = "neur"
if tissue == "conn":
    rm = "RM2"
elif tissue == "neur":
    rm = "RM1"

tp = tp[tp.obs["Sample"].isin([rm, "MR1"])]
tp.obs["Environment"] = ["Mouse-like env." if x == rm else "Rat-like env." for x in list(tp.obs["Sample"])]
tp2 = tp[(tp.obs["Sample"].isin([rm])) & (tp.obs["Species"].isin(["Mouse"]))]
tp2.obs["Environment"] = np.repeat("Host", tp2.obs.shape[0])
tp = tp.concatenate([tp2])
tp2 = tp[(tp.obs["Sample"].isin(["MR1"])) & (tp.obs["Species"].isin(["Rat"]))]
tp2.obs["Environment"] = np.repeat("Host", tp2.obs.shape[0])
tp = tp.concatenate([tp2])



In [ ]:
#Changed gene and cell_type as appropriate

gene = "Grb10"
cell_type = "brain.GABA.neu"
tpx = pd.DataFrame(tp.X.todense())
tpx.index = tp.obs.index
tpx.columns = tp.var.index
tpx = tpx.join(tp.obs)

sns.set(font_scale=1.5)
sns.set_style("white")
sns.swarmplot(data = tpx[tpx[gene] > 0], y = gene, x = "Environment", hue = "Species", palette = {"Mouse":mouse, "Rat":rat}, alpha = 1, order = ["Host", "Rat-like env.", "Mouse-like env."], dodge = True)
plt.ylabel("Normalized counts per cell")
#plt.xlabel("Chimera (donor" + r'$\rightarrow$' + "host)")
plt.xlabel("Extrinsic environment")
plt.title("Expression of $\it{" + gene + "}$ in " + cell_type)
xmin, xmax, ymin, ymax = plt.axis()

#plt.ylim(0, 1)
plt.axvline(0.5, color = "black", linewidth = 2.5, alpha = 1, linestyle="dashed")
#plt.legend(bbox_to_anchor=(1.375, 1.05))
plt.legend([],[], frameon=False)
plt.ylim(0, ymax)

c = -0.33
for i in ["Host", "Rat-like env.", "Mouse-like env."]:
    
    for j in ["Mouse", "Rat"]:
        tpx_cur = tpx[(tpx["Environment"].isin([i])) & (tpx["Species"].isin([j]))]
        tot = tpx_cur.shape[0]
        express = tpx_cur[tpx_cur[gene] > 0].shape[0]
        perc = int(round(express/tot, 2)*100)
        plt.text(c, 0.125, str(perc) + "%")
        c += 0.4
    c += 0.21
    

plt.show()

In [ ]:
#Alternative code to make half-violin/half stripplot
sns.set(font_scale=1.5)
sns.set_style("white")
ax = sns.violinplot(data = tpx, y = gene, x = "Environment", hue = "Species", palette = {"Mouse":mouse, "Rat":rat}, alpha = 1, order = ["Host", "Rat-like env.", "Mouse-like env."], dodge = True)
plt.ylabel("Normalized counts per cell")
#plt.xlabel("Chimera (donor" + r'$\rightarrow$' + "host)")
plt.xlabel("Extrinsic environment")
plt.title("Expression of $\it{" + gene + "}$ in " + cell_type)
xmin, xmax, ymin, ymax = plt.axis()

#plt.ylim(0, 1)
plt.axvline(0.5, color = "black", linewidth = 2.5, alpha = 1, linestyle="dashed")
#plt.legend(bbox_to_anchor=(1.375, 1.05))

xlim = ax.get_xlim()
ylim = ax.get_ylim()
for violin in ax.collections:
    bbox = violin.get_paths()[0].get_extents()
    x0, y0, width, height = bbox.bounds
    violin.set_clip_path(plt.Rectangle((x0, y0), width / 2, height, transform=ax.transData))

old_len_collections = len(ax.collections)
sns.stripplot(data = tpx, y = gene, x = "Environment", hue = "Species", palette = {"Mouse":mouse, "Rat":rat}, alpha = 1, order = ["Host", "Rat-like env.", "Mouse-like env."], dodge = True)
for dots in ax.collections[old_len_collections:]:
    dots.set_offsets(dots.get_offsets() + np.array([0.12, 0]))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.legend([],[], frameon=False)
plt.show()